In [9]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, root_mean_squared_error

print("--- STRATEGY 4: PHYSICS-INFORMED FEATURE ENGINEERING ---")

# --- Step 1: Load Data ---
df = pd.read_csv('..\data\df_recent_enriched.csv', index_col='Date', parse_dates=True)

# --- Step 2: Create Physics-Informed Features ---
print("\n--- Step 2: Engineering new physics-based features ---")

# Feature 1: Dew Point Temperature (using a common approximation)
# This combines temperature and RH into a measure of absolute humidity.
a = 17.625
b = 243.04
gamma = (a * df['Min_Temp_C']) / (b + df['Min_Temp_C']) + np.log(df['RH_mean'] / 100.0)
df['Dew_Point_C'] = (b * gamma) / (a - gamma)
print("✅ Created 'Dew_Point_C' feature.")

# Feature 2: Clear Sky Index (Ratio of actual to theoretical max solar radiation)
# We can approximate the "clear sky potential" using our seasonal Day_of_Year_sin feature.
# We scale the sin wave to roughly match the bounds of Solar_Rad_sum.
max_solar = df['Solar_Rad_sum'].max()
min_solar = df['Solar_Rad_sum'].min()
# Create a theoretical clear-sky radiation curve based on the day of the year
df['Theoretical_Solar_Rad'] = min_solar + (max_solar - min_solar) * (df['Day_of_Year_sin'] + 1) / 2
# The index is a powerful feature representing "cloudiness"
df['Clear_Sky_Index'] = df['Solar_Rad_sum'] / (df['Theoretical_Solar_Rad'] + 1e-6) # Add epsilon to avoid division by zero
print("✅ Created 'Clear_Sky_Index' feature.")

# Feature 3: Heat Index (a simple version for demonstration)
# This captures the non-linear interaction between heat and humidity.
# A proper formula is complex, but we can create a simple interaction term.
df['Heat_Index_Interaction'] = df['Max_Temp_C'] * df['RH_mean']
print("✅ Created 'Heat_Index_Interaction' feature.")


# --- Step 3: Build the Physics-Aware Model ---
print("\n--- Step 3: Building the final, physics-aware model ---")

# Our feature set is now smarter and more focused.
# We will include our new physics features and the original atmospheric variables.
feature_cols = [
    'Max_Temp_C', 'Min_Temp_C', 'Precipitation_mm', # Use current day's actuals
    'RH_mean', 'Wind_Speed_ms_mean', 'Solar_Rad_sum', # Key atmospheric drivers
    'Dew_Point_C', 'Clear_Sky_Index', 'Heat_Index_Interaction' # Our powerful new features
]

# Create targets and define X, y
for horizon in range(1, 4):
    df[f'Target_Max_Temp_C_t+{horizon}'] = df['Max_Temp_C'].shift(-horizon)
    df[f'Target_Min_Temp_C_t+{horizon}'] = df['Min_Temp_C'].shift(-horizon)
df.dropna(subset=[f'Target_Max_Temp_C_t+3'] + feature_cols, inplace=True)

target_cols = [col for col in df.columns if 'Target_' in str(col)]
X = df[feature_cols]
y = df[target_cols]

# Chronological split
split_index = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

# Use the simple, regularized XGBoost parameters we know work best for this dataset
simple_xgb_params = {
    'n_estimators': 150, 'learning_rate': 0.05, 'max_depth': 4,
    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 1, 'reg_alpha': 0.1,
    'objective': 'reg:absoluteerror', 'random_state': 42, 'n_jobs': -1
}
physics_model = xgb.XGBRegressor(**simple_xgb_params)
physics_model.fit(X_train, y_train)

# --- Step 4: Evaluate the Physics-Aware Model ---
print("\n--- FINAL PHYSICS-AWARE MODEL PERFORMANCE ---")
y_pred_physics = physics_model.predict(X_test)
y_pred_physics_df = pd.DataFrame(y_pred_physics, index=X_test.index, columns=y_test.columns)

for i in range(1, 4):
    mae_max = mean_absolute_error(y_test[f'Target_Max_Temp_C_t+{i}'], y_pred_physics_df[f'Target_Max_Temp_C_t+{i}'])
    rmse_max = root_mean_squared_error(y_test[f'Target_Max_Temp_C_t+{i}'], y_pred_physics_df[f'Target_Max_Temp_C_t+{i}'])
    mae_min = mean_absolute_error(y_test[f'Target_Min_Temp_C_t+{i}'], y_pred_physics_df[f'Target_Min_Temp_C_t+{i}'])
    rmse_min = root_mean_squared_error(y_test[f'Target_Min_Temp_C_t+{i}'], y_pred_physics_df[f'Target_Min_Temp_C_t+{i}'])
    print(f"\nHorizon t+{i}:")
    print(f"  Max Temp - MAE: {mae_max:.2f}, RMSE: {rmse_max:.2f}")
    print(f"  Min Temp - MAE: {mae_min:.2f}, RMSE: {rmse_min:.2f}")

--- STRATEGY 4: PHYSICS-INFORMED FEATURE ENGINEERING ---

--- Step 2: Engineering new physics-based features ---
✅ Created 'Dew_Point_C' feature.
✅ Created 'Clear_Sky_Index' feature.
✅ Created 'Heat_Index_Interaction' feature.

--- Step 3: Building the final, physics-aware model ---

--- FINAL PHYSICS-AWARE MODEL PERFORMANCE ---

Horizon t+1:
  Max Temp - MAE: 1.54, RMSE: 2.15
  Min Temp - MAE: 0.92, RMSE: 1.18

Horizon t+2:
  Max Temp - MAE: 1.75, RMSE: 2.41
  Min Temp - MAE: 1.10, RMSE: 1.44

Horizon t+3:
  Max Temp - MAE: 1.90, RMSE: 2.58
  Min Temp - MAE: 1.19, RMSE: 1.54
